第二章　自然言語と単語の分散表現　メモ

In [39]:
text = 'You say goodbay and I say hello.'
text = text.lower()
print(text)
text.replace('.', ' .')

you say goodbay and i say hello.


'you say goodbay and i say hello .'

In [40]:
words = text.split(' ')
print(words)

['you', 'say', 'goodbay', 'and', 'i', 'say', 'hello.']


In [41]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id :
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

print(id_to_word[1])
print(word_to_id['say'])

say
1


In [42]:
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
print(corpus)

[0 1 2 3 4 1 5]


In [43]:
def preprocess(text):
    text = text.lower()
    text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id :
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    
#     単語IDのリスト
    corpus = [word_to_id[w] for w in words]
     
    return corpus, word_to_id, id_to_word

目的：コーパスを使って単語の意味を抽出する

In [44]:
text = 'You say goodbye adn I say hello. '
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus)
print(id_to_word)

[0, 1, 2, 3, 4, 1, 5, 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'adn', 4: 'i', 5: 'hello.', 6: ''}


In [48]:
# コーパスから共起行列を作成する
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
                
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] +=1

    return co_matrix

In [49]:
#  コサイン類似度（cosine similarity）

def cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x**2)) + eps)
    ny = y / (np.sqrt(np.sum(y**2)) + eps)
    
    return np.dot(nx,ny)

In [52]:
text = 'You say goodbay and I say hello.'
copus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(copus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)

# c0 = C[word_to_id['you']]
# c1 = C[word_to_id['i']]
# print(cos_similarity(c0, c1))


[query]you
 hello.: 0.9999999800000005
 goodbay: 0.7071067691154799
 i: 0.7071067691154799
 say: 0.0
 and: 0.0


In [51]:
# クエリに対して類似した単語を上位から順に表示する関数
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    
    if query not in word_to_id:
        print('%s is no found' % query)
        return
    
    print('\n[query]' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
    
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))
        
        count += 1
        if count >= top:
            return
        